In [ ]:
import torch
import pandas as pd
import numpy as np
# 新数据
# 读取所有数据
paths = [
    'C:/Users/Administrator/Desktop/My research/2_ML_predict_product/data_product/1500K_3atm/clean_processed_data_1500_1.csv',
    'C:/Users/Administrator/Desktop/My research/2_ML_predict_product/data_product/1500K_3atm/clean_processed_data_1500_2.csv',
    'C:/Users/Administrator/Desktop/My research/2_ML_predict_product/data_product/1500K_3atm/clean_processed_data_1500_3.csv',
    'C:/Users/Administrator/Desktop/My research/2_ML_predict_product/data_product/1500K_3atm/clean_processed_data_1500_4.csv',
    'C:/Users/Administrator/Desktop/My research/2_ML_predict_product/data_product/1500K_3atm/clean_processed_data_1500_5.csv',
    'C:/Users/Administrator/Desktop/My research/2_ML_predict_product/data_product/1500K_3atm/clean_processed_data_1500_6.csv'
]

# 读取并拼接数据
# all_data = pd.concat([pd.read_csv(path) for path in paths], ignore_index=True)
# all_data = np.array(all_data)
# all_data.shape

In [ ]:
all_data = []
for path in paths:
    data = pd.read_csv(path)
    data = np.array(data)
    all_data.append(data)
all_data = np.concatenate(all_data, axis=0)

In [ ]:
all_data.shape

In [ ]:
# 
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
import pandas as pd
# 设定随机种子以保证结果可复现
torch.manual_seed(0)
np.random.seed(0)

# 设定设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 处理数据的函数
def process_data(data):
    pro_data = []
    for record in data:
        # 每条记录是一个字符串，将其按分号分割
        elements = record[0].split(';')
        processed_data = []
        for element in elements:
            # print(element)
            # 去除括号并按逗号分割成元素和数字
            mol, count = element.strip('()').split(',')
            processed_data.append((mol, int(count)))
        pro_data.append(np.array(processed_data))
    return pro_data

# 调用函数处理数据
processed = process_data(all_data) 
processed = np.array(processed, dtype=object)
print(processed.shape)

# 对原始数据(processed)继续进行统计分析

In [ ]:
# 统计所有物种类型
unique_species = set()

for record in processed:
    for element in record:
        # element 是一个包含 ['物种名称', '停留时间'] 的数组
        mol = element[0]  # 获取物种名称
        unique_species.add(mol)

# 输出所有物种类型的名称
print(len(unique_species))
#print(unique_species)


In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

def count_species_with_time(processed, n):
    """
    统计每个物种的总出现次数以及停留时间小于等于n的次数。

    参数:
    - processed (np.array): 处理后的数据数组，元素为每条记录的物种及停留时间。
    - n (int): 停留时间的阈值。

    返回:
    - species_counter (Counter): 每个物种的总出现次数。
    - species_counter_le_n (Counter): 每个物种停留时间小于等于n的出现次数。
    """
    species_counter = Counter()
    species_counter_le_n = Counter()
    
    for record in processed:
        for element in record:
            mol = element[0]
            try:
                count = int(element[1])
            except ValueError:
                print(f"无法将停留时间转换为整数: {element[1]}，跳过该元素。")
                continue  # 跳过无法转换的元素
            
            species_counter[mol] += 1
            if count <= n:
                species_counter_le_n[mol] += 1
    
    return species_counter, species_counter_le_n

def print_species_statistics(species_counts, species_counts_le_n, n):
    """
    打印每种物种的总出现次数和停留时间小于等于n的出现次数，按总出现次数降序排列。

    参数:
    - species_counts (Counter): 每个物种的总出现次数。
    - species_counts_le_n (Counter): 每个物种停留时间小于等于n的出现次数。
    - n (int): 停留时间的阈值。
    """
    # 获取按总出现次数降序排列的物种列表
    sorted_species = species_counts.most_common()
    
    # 打印表头
    print(f"\n{'物种类型':<20} {'总出现次数':<15} {'停留时间<= ' + str(n) + ' 次数':<20}")
    print("-" * 55)
    
    # 打印每种物种的统计数据
    for mol, total in sorted_species:
        le_n = species_counts_le_n.get(mol, 0)
        print(f"{mol:<20} {total:<15} {le_n:<20}")



# 示例调用
if __name__ == "__main__":

    
    n = 5  # 设定阈值
    
    # 统计物种出现次数和停留时间小于等于n的出现次数
    species_counts, species_counts_le_n = count_species_with_time(processed, n)
    
    # 打印统计结果，按总出现次数降序排列
    print_species_statistics(species_counts, species_counts_le_n, n)


In [ ]:
from collections import Counter, defaultdict
# -----------------------------------------------
# 统计每个物种的最大、最小和平均停留时间
# -----------------------------------------------

# 使用 defaultdict 存储每个物种的最小、最大停留时间及总停留时间
species_time_stats = defaultdict(lambda: {'min': None, 'max': None, 'total': 0, 'count': 0})

for record in processed:
    for element in record:
        mol, count = element[0], int(element[1])
        if species_time_stats[mol]['min'] is None or count < species_time_stats[mol]['min']:
            species_time_stats[mol]['min'] = count
        if species_time_stats[mol]['max'] is None or count > species_time_stats[mol]['max']:
            species_time_stats[mol]['max'] = count
        species_time_stats[mol]['total'] += count
        species_time_stats[mol]['count'] += 1

# 计算平均停留时间并准备输出列表
species_time_list = sorted(
    [
        (
            stats['total'] / stats['count'] if stats['count'] > 0 else 0,
            stats['max'],
            stats['min'],
            mol
        )
        for mol, stats in species_time_stats.items()
    ],
    key=lambda x: x[0]
)

# 输出统计结果
print("\n每种物种类型的最大、最小和平均停留时间：")
for avg_time, max_time,min_time,mol in species_time_list:
    print(f"{mol}: 最小停留时间 = {min_time}, 最大停留时间 = {max_time}, 平均停留时间 = {avg_time:.2f}")





# 对原始数据(processed)继续进行深度处理

In [ ]:
import numpy as np

def filter_and_merge_one_record(record, min_time=5):
    """
    对单条记录进行过滤和合并操作：
    1. 删除停留时间小于 min_time 的状态。
    2. 将删除后连续且相同物种的状态合并为一个状态（停留时间累加）。
    
    参数：
    - record: 形如 array([['MoO','33'], ['MoS','251'], ['MoS','254'], ...]) 的数组
    - min_time: 停留时间的阈值，删除停留时间小于该值的状态
    
    返回：
    - 处理后的 NumPy 数组，满足以上两步操作
    """
    # 1) 过滤：删除停留时间 < min_time 的状态
    filtered = [row for row in record if int(row[1]) >= min_time]
    
    # 2) 合并连续相同物种的状态
    merged = []
    for mol, time_str in filtered:
        time_val = int(time_str)
        if merged and merged[-1][0] == mol:
            # 若与上一个物种相同，则合并（累加停留时间）
            merged[-1][1] = str(int(merged[-1][1]) + time_val)
        else:
            # 否则添加新的物种状态
            merged.append([mol, str(time_val)])
    
    return np.array(merged)

def filter_and_merge_processed(processed, min_time=5):
    """
    对完整的 processed 数据进行过滤和合并操作：
    对 processed 每个元素（单条记录）调用 filter_and_merge_one_record
    
    参数：
    - processed: 形如 [record_1, record_2, ...] 的对象数组（dtype=object）
    - min_time: 停留时间的阈值
    
    返回：
    - 处理后的 NumPy 对象数组
    """
    new_processed = []
    for idx, record in enumerate(processed):
        merged_record = filter_and_merge_one_record(record, min_time=min_time)
        new_processed.append(merged_record)
    
    return np.array(new_processed, dtype=object)

# -------------------- 测试示例 --------------------
if __name__ == "__main__":    
    # 设定阈值，比如 n=5
    n = 5
    
    # 调用过滤和合并函数
    new_processed = filter_and_merge_processed(processed, min_time=n)
    
    # 打印过滤前和过滤后的结果
    print("过滤前 (原始数据) ：")
    print(processed[0])
    
    print("\n过滤后 (删除停留时间 < 5 并合并相邻相同状态) ：")
    print(new_processed[0][:20])


# 数据编码操作

In [ ]:
import numpy as np
# 编码，转换成向量
def incode(molecule):
    vec = np.zeros(3)
    length = len(molecule)
    Mo = molecule.find('Mo')
    O = molecule.find('O')
    S = molecule.find('S')
    # print(Mo, O, S)
    moi = oi = si = 0
    moi = 1
    if O==-1 and S==-1 and length>2:
        moi = molecule[Mo+2:]
    elif O != -1:
        if O > Mo + 2:
            moi = int(molecule[Mo+2:O])
        else:
            moi = 1
        if S == O + 1 or O+1==length:
            oi = 1
        elif S==-1:
            oi = int(molecule[O+1:])
        else:
            oi = int(molecule[O+1:S])
    elif S != -1:
        if S > Mo + 2:
            moi = int(molecule[Mo+2:S])
        else:
            moi = 1
        if S+1==length:
            si = 1
        else:
            si = int(molecule[S+1:])
    if S!=-1:
        if S+1==length:
            si = 1
        else:
            si = int(molecule[S+1:])
    vec = np.array([moi, oi, si])
    return vec
incode('MoOS13')